In [220]:
import matplotlib.pyplot as plt
from funcs import *
import os

os.chdir(R'C:\Users\adria\Desktop\STUDIA_FOLDERY\TELEDETEKCJA')

In [221]:
cir_raster = r"TIFs\CIR_2015.tif"
cir_dataset = read_spatial_raster(cir_raster)

rgb_raster = r"TIFs\RGB_2015.tif"
rgb_dataset = read_spatial_raster(rgb_raster)

band_nir = cir_dataset.GetRasterBand(1)
array_nir = band_nir.ReadAsArray()
array_nir = np.copy(array_nir)

band_red = cir_dataset.GetRasterBand(2)
array_red = band_red.ReadAsArray()
array_red = np.copy(array_red)

band_green = cir_dataset.GetRasterBand(3)
array_green = band_green.ReadAsArray()
array_green = np.copy(array_green)

band_blue = rgb_dataset.GetRasterBand(3)
array_blue = band_blue.ReadAsArray()
array_blue = np.copy(array_blue)

c:\Users\adria\anaconda3\envs\teledetekcja\Lib\site-packages\osgeo\gdal.py:8817: RuntimeWarning: CIR_2015.tif: JPEGFixupTagsSubsamplingSec:Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [1,1]
  return _gdal.Open(*args)
c:\Users\adria\anaconda3\envs\teledetekcja\Lib\site-packages\osgeo\gdal.py:8817: RuntimeWarning: RGB_2015.tif: JPEGFixupTagsSubsamplingSec:Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [1,1]
  return _gdal.Open(*args)


In [222]:
ids = np.array([7, 0, 3])

vector_features = gpd.read_file(R"budynki\budynki.shp")

features = reproject_geodataframe(vector_features, cir_dataset.GetProjection())
features = convert_to_pixel_system(features, cir_dataset.GetGeoTransform())

## kanały

In [223]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  
    
    print(f"Obiekt {id}")
    
    # nir
    masked_fragment = np.copy(fragment_nir)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_nir[~no_data_mask]  # Select just valid pixels
    print(f"NIR: min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\nir_budynek{id}.png", masked_fragment, cmap='gray')

    # red
    masked_fragment = np.copy(fragment_red)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_red[~no_data_mask]
    print(f"Red min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\red_budynek{id}.png", masked_fragment, cmap='gray')

    # green
    masked_fragment = np.copy(fragment_green)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_green[~no_data_mask]
    print(f"Green min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\green_budynek{id}.png", masked_fragment, cmap='gray')

    # blue
    masked_fragment = np.copy(fragment_blue)
    masked_fragment[no_data_mask] = 0

    pixel_values = fragment_blue[~no_data_mask] 
    print(f"Blue min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\blue_budynek{id}.png", masked_fragment, cmap='gray')

Obiekt 7
NIR: min: 48, max: 157, mean: 77.2828999211978
Red min: 40, max: 131, mean: 70.44365642237983
Green min: 43, max: 134, mean: 71.22695035460993
Blue min: 37, max: 120, mean: 65.39322301024428
Obiekt 0
NIR: min: 39, max: 187, mean: 101.36163265306122
Red min: 47, max: 203, mean: 127.01306122448979
Green min: 60, max: 217, mean: 132.78448979591838
Blue min: 43, max: 205, mean: 128.09142857142857
Obiekt 3
NIR: min: 59, max: 178, mean: 90.41953488372093
Red min: 60, max: 189, mean: 98.90325581395349
Green min: 55, max: 193, mean: 96.62139534883721
Blue min: 44, max: 191, mean: 92.05953488372093


## kanały bez maski

In [224]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  
    
    print(f"Obiekt {id}")
    
    # nir
    masked_fragment = np.copy(fragment_nir)
    pixel_values = fragment_nir[~no_data_mask]  # Select just valid pixels
    print(f"NIR: min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\nir_budynek{id}_bezmaski.png", masked_fragment, cmap='gray')

    # red
    masked_fragment = np.copy(fragment_red)
    pixel_values = fragment_red[~no_data_mask]
    print(f"Red min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\red_budynek{id}_bezmaski.png", masked_fragment, cmap='gray')

    # green
    masked_fragment = np.copy(fragment_green)
    pixel_values = fragment_green[~no_data_mask]
    print(f"Green min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\green_budynek{id}_bezmaski.png", masked_fragment, cmap='gray')

    # blue
    masked_fragment = np.copy(fragment_blue)
    pixel_values = fragment_blue[~no_data_mask] 
    print(f"Blue min: {pixel_values.min()}, max: {pixel_values.max()}, mean: {pixel_values.mean()}")

    plt.imsave(f"tex\\spektralne\\blue_budynek{id}_bezmaski.png", masked_fragment, cmap='gray')

Obiekt 7
NIR: min: 48, max: 157, mean: 77.2828999211978
Red min: 40, max: 131, mean: 70.44365642237983
Green min: 43, max: 134, mean: 71.22695035460993
Blue min: 37, max: 120, mean: 65.39322301024428
Obiekt 0
NIR: min: 39, max: 187, mean: 101.36163265306122
Red min: 47, max: 203, mean: 127.01306122448979
Green min: 60, max: 217, mean: 132.78448979591838
Blue min: 43, max: 205, mean: 128.09142857142857
Obiekt 3
NIR: min: 59, max: 178, mean: 90.41953488372093
Red min: 60, max: 189, mean: 98.90325581395349
Green min: 55, max: 193, mean: 96.62139534883721
Blue min: 44, max: 191, mean: 92.05953488372093


## kompozycje

In [225]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    # rgb
    rgb = np.dstack([fragment_red, fragment_green, fragment_blue])
    # rgb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\rgb_budynek{id}.png", rgb, cmap='gray')

    # irgb
    irgb = np.dstack([fragment_nir, fragment_green, fragment_blue])
    # irgb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\irgb_budynek{id}.png", irgb, cmap='gray')

    # irrb
    irrb = np.dstack([fragment_nir, fragment_red, fragment_blue])
    # irrb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\irrb_budynek{id}.png", irrb, cmap='gray')

    # rgir
    rgir = np.dstack([fragment_red, fragment_green, fragment_nir])
    # rgir[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\rgir_budynek{id}.png", rgir, cmap='gray')

## kompozycje bez maski

In [226]:
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    # rgb
    rgb = np.dstack([fragment_red, fragment_green, fragment_blue])
    rgb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\rgb_budynek{id}_maska.png", rgb, cmap='gray')

    # irgb
    irgb = np.dstack([fragment_nir, fragment_green, fragment_blue])
    irgb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\irgb_budynek{id}_maska.png", irgb, cmap='gray')

    # irrb
    irrb = np.dstack([fragment_nir, fragment_red, fragment_blue])
    irrb[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\irrb_budynek{id}_maska.png", irrb, cmap='gray')

    # rgir
    rgir = np.dstack([fragment_red, fragment_green, fragment_nir])
    rgir[no_data_mask] = 0

    plt.imsave(f"tex\\spektralne\\rgir_budynek{id}_maska.png", rgir, cmap='gray')

## wskaźniki 2015

In [227]:
def calculate_NDVI(nir_array, red_array):
    IR = np.float32(nir_array)
    R = np.float32(red_array)
    NDVI = (IR-R)/(IR+R)
    NDVI[np.isnan(NDVI)] = 0
    return NDVI

def calculate_MSAVI(nir_array, red_array): 
    IR = np.ma.masked_invalid(np.float32(nir_array / 255.0))
    R = np.ma.masked_invalid(np.float32(red_array / 255.0))
    
    term = (2 * IR + 1)**2 - 8 * (IR - R)
    term[term < 0] = 0 
    
    MSAVI = (2 * IR + 1 - np.sqrt(term)) / 2
    MSAVI = np.ma.filled(MSAVI, 0) 
    
    MSAVI[MSAVI < -1] = -1
    MSAVI[MSAVI > 1] = 1
    
    return MSAVI

print('2015\n')
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    print(f"Obiekt {id}")

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    ndvi = calculate_NDVI(fragment_nir, fragment_red)
    plt.imshow(ndvi, cmap='gray')
    cbar = plt.colorbar()
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f"tex\\spektralne\\ndvi_budynek{id}_2015.png", bbox_inches='tight')
    plt.clf()
    print(f"NDVI min: {ndvi.min()}, max: {ndvi.max()}, mean: {ndvi.mean()}")
    # plt.imsave(f"tex\\spektralne\\ndvi_budynek{id}_2015.png", ndvi, cmap='gray')

    msavi = calculate_MSAVI(fragment_nir, fragment_red)
    plt.imshow(msavi, cmap='gray')
    cbar = plt.colorbar()
    plt.xticks([])
    plt.yticks([])
    plt.savefig(f"tex\\spektralne\\msavi_budynek{id}_2015.png", bbox_inches='tight')
    plt.clf()
    print(f"MSAVI min: {msavi.min()}, max: {msavi.max()}, mean: {msavi.mean()}")
    # plt.imsave(f"tex\\spektralne\\bi_budynek{id}_2015.png", bi, cmap='gray')

2015

Obiekt 7
NDVI min: -0.07500000298023224, max: 0.29323309659957886, mean: 0.08258163928985596
MSAVI min: -0.061637389159784894, max: 0.22657239371055737, mean: 0.0677343479487076
Obiekt 0
NDVI min: -0.23880596458911896, max: 0.25641027092933655, mean: -0.03661259263753891
MSAVI min: -0.1674495990703857, max: 0.200000006097094, mean: -0.036404144634515266
Obiekt 3
NDVI min: -0.13812154531478882, max: 0.22274881601333618, mean: 0.028374115005135536
MSAVI min: -0.11496391772533754, max: 0.2039024084563429, mean: 0.02572976924411628


<Figure size 640x480 with 0 Axes>

## rastry 2023

In [228]:
cir_raster = r"TIFs\CIR_2023.tif"
cir_dataset = read_spatial_raster(cir_raster)

rgb_raster = r"TIFs\RGB_2023.tif"
rgb_dataset = read_spatial_raster(rgb_raster)

band_nir = cir_dataset.GetRasterBand(1)
array_nir = band_nir.ReadAsArray()
array_nir = np.copy(array_nir)

band_red = cir_dataset.GetRasterBand(2)
array_red = band_red.ReadAsArray()
array_red = np.copy(array_red)

band_green = cir_dataset.GetRasterBand(3)
array_green = band_green.ReadAsArray()
array_green = np.copy(array_green)

band_blue = rgb_dataset.GetRasterBand(3)
array_blue = band_blue.ReadAsArray()
array_blue = np.copy(array_blue)

ids = np.array([7, 0, 3])

vector_features = gpd.read_file(R"budynki\budynki.shp")

features = reproject_geodataframe(vector_features, cir_dataset.GetProjection())
features = convert_to_pixel_system(features, cir_dataset.GetGeoTransform())

## wskaźniki 2023

In [229]:
print('2023\n')
for id in ids:
    example_feature = features.iloc[id]
    example_feature = example_feature['geometry']

    bounds = example_feature.bounds
    bounds = np.float64(bounds)

    bounds[:2] = np.floor(bounds[:2])
    bounds[2:] = np.ceil(bounds[2:])
    bounds = np.int64(bounds)

    fragment_nir = array_nir[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_red = array_red[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_green = array_green[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    fragment_blue = array_blue[
        bounds[1]: bounds[3],
        bounds[0]: bounds[2]
    ]

    feature_in_fragment_frame = shapely.affinity.translate(example_feature, -bounds[0], -bounds[1])

    from rasterio.features import rasterize

    print(f"Obiekt {id}")

    # maska
    no_data_mask = rasterize([feature_in_fragment_frame], fragment_nir.shape)
    no_data_mask = np.bool_(no_data_mask)
    no_data_mask = ~no_data_mask  

    ndvi = calculate_NDVI(fragment_nir, fragment_red)
    plt.imshow(ndvi, cmap='gray')
    cbar = plt.colorbar()
    plt.xticks([])
    plt.yticks([])
    print(f"NDVI min: {ndvi.min()}, max: {ndvi.max()}, mean: {ndvi.mean()}")
    plt.savefig(f"tex\\spektralne\\ndvi_budynek{id}_2023.png", bbox_inches='tight')
    plt.clf()

    msavi = calculate_MSAVI(fragment_nir, fragment_red)
    plt.imshow(msavi, cmap='gray')
    cbar = plt.colorbar()
    plt.xticks([])
    plt.yticks([])
    print(f"MSAVI min: {msavi.min()}, max: {msavi.max()}, mean: {msavi.mean()}")
    plt.savefig(f"tex\\spektralne\\msavi_budynek{id}_2023.png", bbox_inches='tight')
    plt.clf()

2023

Obiekt 7
NDVI min: -0.7599999904632568, max: 0.6056337952613831, mean: -0.06239995360374451
MSAVI min: -0.3319346786405236, max: 0.4484128392691128, mean: 0.009091574737330865
Obiekt 0
NDVI min: -1.0, max: 0.5740740895271301, mean: -0.16245964169502258
MSAVI min: -0.3250841649100362, max: 0.37707692350225724, mean: -0.08611699057253473
Obiekt 3
NDVI min: -1.0, max: 0.27272728085517883, mean: -0.17590896785259247
MSAVI min: -0.32625915325578847, max: 0.22385356551117563, mean: -0.09957049997339108


<Figure size 640x480 with 0 Axes>